<a href="https://colab.research.google.com/github/shstreuber/AI/blob/main/Week14_CS_345_545_AI_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#**0. What Is A Chatbot?**

AI Chatbots are applications that businesses and other organizations use to automate conversations between AI and humans. These conversations can occur through text (as with Copilot) or speech (as with Alexa, for example). Chatbots must comprehend and imitate human conversation when engaging with users.

In order to process a large amount of natural language data, an AI uses NLP or Natural Language Processing. NLP tasks involve breaking down human text and audio signals from voice data in ways that computers can analyze and convert into comprehensible data. Some of the tasks in NLP data ingestion include:

1. **Speech Recognition,** which involves converting speech into text using a subprocess called speech tagging, which allows a computer to break down speech and add context, accents, or other speech attributes.
2. **Word Sense Disambiguation,** which selects the most appropriate meaning for a word based on its context. For instance, it helps determine whether a word functions as a verb or a pronoun.
3. **Named Entity Recognition (NER),** Nwhic identifies words and phrases as specific entities, such as recognizing “Dav” as a person’s name or “America” as thase name of a country.
4. **Sentiment Analysis,** which focuses on extracting nuances and hidden emotions, like attitude, sarcasm, fear, or joy, from human speech.

There are essentially two types of chatbots:
1. **Rule-Based (Scripted) Chatbots,** which operate based on predetermined scripts stored in their library. When a user inputs a query, the chatbot replies according to the predefined script within its library. One drawback of this type of chattbot is that users must structure their queries very precisely.
2. **Self-Learning (Artificially Intelligent) Chatbots,** which rely on a combination of NLP (for analysis) and AI in order to respond to nuanced questions and learn from each interaction to provide improved responses in the future.

And then a combination of the two.

#**1. Basic Text Input Chatbot**

Read [this article on Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/10/complete-guide-to-build-your-ai-chatbot-with-nlp-in-python/). Here, the author is building a speech-to-text chatbot natively on Microsoft Windows. To get the code from this post working in Google Colab, we have to make some adjustments since there's an issue with building the PyAudio wheel in Google Colab. That is because, unfortunately, Google Colab does not support building packages that require compiling C extensions, such as PyAudio. That works only in MS Windows. I learned that the hard way.

To rewrite the code so that it runs in Google Colab without requiring PyAudio or needing to connect to your local microphone, we will replace the speech-to-text functionality with text input.

**NOTE:** The code below  generates a right-padding warning (which sent me down a fruitless hours-long debugging path) but works as intended by carrying on a conversation.

First, we import all the libraries we need

In [ ]:
import os
import numpy as np
import datetime
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

To ensure that all libraries are compatible with Python 3.10, we are going to do a few checks now. We need transformers >= 4.12 and torch >= 1.10 (or higher)

In [ ]:
pip show transformers torch

Next, we build a ChatBot class to provide functionality for initializing the chatbot, getting user input, responding to user input, checking for specific commands, and providing the current time.:

1. Initialization (__init__ method):
When you create a new instance of the ChatBot class, it initializes with a specified name. This name is printed to indicate that the chatbot is starting up.
2. Getting User Input (get_text_input method):
This method prompts the user to enter text input, and it returns the input provided by the user.
3. Text Output (text_to_speech method):
This method takes a text input and prints it to the console, simulating the chatbot's response.
4. Wake-Up Check (wake_up method):
This method checks if the chatbot's name (converted to lowercase) is mentioned in the text input. If it is, it returns True; otherwise, it returns False.
5. Time Action (action_time method):
This method retrieves the current time and returns it in the format "HH:MM".








In [ ]:
# ChatBot class
class ChatBot:
    def __init__(self, name="ChatBot"):
        print("----- Starting up", name, "-----")
        self.name = name.lower()

    def get_text_input(self):
        return input("You --> ")

    @staticmethod
    def text_to_speech(text: str):
        print("ChatBot -->", text)

    def wake_up(self, text: str) -> bool:
        return self.name in text.lower()

    @staticmethod
    def action_time() -> str:
        return datetime.datetime.now().strftime('%H:%M')

Next, we build a response generator function.

The **`generate_response`** function processes user input to generate a contextually relevant response using the `DialoGPT` model. It first tokenizes the user's input and appends it to the conversation history (if any). The function then generates a response by passing the combined input and history to the model, using parameters like `top_k`, `top_p`, and `temperature` to control the randomness and diversity of the response. After generating the output tokens, the function decodes them into human-readable text and returns both the response and the updated conversation history, ensuring that future responses maintain the context of the entire dialogue. This allows the chatbot to provide coherent, conversational replies.

In [ ]:
# Response generator function
def generate_response(user_input, chat_history_ids, tokenizer, model, device):
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt').to(device)
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.75,
    )

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response, chat_history_ids

To run the chatbot, we need to initialize the chatbot, load the necessary components (tokenizer and conversational pipeline), greet the user, handle user input, process commands, engage in conversation, and exit gracefully when prompted.

**NOTE**: When starting up the chatbot, you will see a `HF_Token` Warning. This warning is telling you that you don’t have the authentication token (`HF_TOKEN`) set up in Colab’s secret storage. Hugging Face allows you to work with models without authentication, but it’s recommended for easier access and use of the Hugging Face Hub. Authentication may be required in certain situations (e.g., for private models or for rate-limited API access). In our case, you don't need to authenticate to use public models like DialoGPT. This warning can be safely ignored unless you're working with private models or want the benefits of authentication.

Here is how all of this magic happens:








In [ ]:
# Main runner
if __name__ == "__main__":
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    chatbot = ChatBot(name="ChatBot")

    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium").to(device)

    chatbot.text_to_speech("Hello, I am ChatBot. How can I assist you today?")

    chat_history_ids = None
    active = True

    while active:
        user_input = chatbot.get_text_input().strip()

        if chatbot.wake_up(user_input):
            response = "Hello, I am ChatBot. How can I assist you today?"

        elif "time" in user_input.lower():
            response = chatbot.action_time()

        elif any(word in user_input.lower() for word in ["thank", "thanks"]):
            response = np.random.choice([
                "You're welcome!", "Anytime!", "No problem!", "Cool!",
                "I'm here if you need me!", "Mention not"
            ])

        elif any(word in user_input.lower() for word in ["exit", "close", "bye"]):
            response = np.random.choice([
                "Tata", "Have a good day", "Bye", "Goodbye", "Hope to meet soon", "Peace out!"
            ])
            active = False

        else:
            try:
                response, chat_history_ids = generate_response(user_input, chat_history_ids, tokenizer, model, device)
            except Exception as e:
                response = f"Oops! Something went wrong: {e}"

        chatbot.text_to_speech(response)

    print("----- Closing down ChatBot -----")

#**2. Chatbot Using OpenAI**
Yes, GPT!

Try the solution in the video out below. I've started you off below with installing the appropriate version of openai

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/q5HiD5PNuck?si=ETpeM3-4NInvR_s9" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture"allowfullscreen></iframe>')

In [ ]:
#!pip install OpenAI
!pip install openai==0.28 # You will need this version of openai to make the code in the video work.

In [ ]:
import openai

Now get a project API key from https://platform.openai.com/api-keys and add it below

In [ ]:
openai.api_key = 'your API key here'

Type here the code from the video and try it out

#**3. A More Advanced Chatbot Using Chatterbot**
This example comes from [hubspot](https://blog.hubspot.com/website/python-ai-chat-bot). The code uses the [Chatterbot AI framework](https://chatterbot.readthedocs.io/en/stable/), a conversational dialog engine. For the Chatterbot Quickstart, check [here](https://chatterbot.readthedocs.io/en/stable/quickstart.html).

***NOTE:*** Implementing Chatterbot on Google Colab used to require rolling back the Google Colab Python version to 3.8-full. It does not any more. A previous version of this notebook contains that code.

In [ ]:
## 1. Installing required libraries--this takes a looooooooong time!

#!pip install pyyaml==5.1.1 # should not be necessary
!pip install chatterbot
!python -m chatterbot --version

# To install the chatterbot corpus, we should be able to use the simple pip command, but need to instead clone the git repo
# !pip install chatterbot-corpus ##--this should work, but doesn't
!git clone https://github.com/gunthercox/chatterbot-corpus.git

Now we are ready to import the libraries we need.

In [ ]:
## 2. Importing required libraries
import os

# Import chatterbot
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer

# Check if the corpus is installed correctly, including the English library
os.listdir('chatterbot-corpus/chatterbot_corpus/data')

In [ ]:
## 2.1 What is in the English library?
!ls -R chatterbot-corpus/chatterbot_corpus/data/english/

In [ ]:
## 3. Initializing the ChatBot
chatbot = ChatBot(
    'ExampleBot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///database.db',  # Use SQLite database to store conversation history
    logic_adapters=[
        'chatterbot.logic.BestMatch',
        'chatterbot.logic.MathematicalEvaluation',  # This allows the bot to handle math-related questions
        'chatterbot.logic.TimeLogicAdapter',  # The bot will be able to tell time
    ],
    preprocessors=['chatterbot.preprocessors.clean_whitespace'],  # Clean the user input
)

In [ ]:
## 4. Creating a new instance of a ChatterBotCorpusTrainer and training the bot with the English corpus
trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train('/content/chatterbot-corpus/chatterbot_corpus/data/english')

In [ ]:
## 5. Function to get a response from the bot
def get_response(user_input):
    return chatbot.get_response(user_input)

And now, we are ready to test our chatbot!

In [ ]:
## 6. Test the chatbot
print("Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Goodbye!")
        break
    response = get_response(user_input)
    print(f"Bot: {response}")

**BUT** that is boring! How about actually training the chatbot with our own data? Here is how we will make that work:

* `ListTrainer`: This trainer allows you to train the chatbot using a list of conversation pairs (like a small dialogue).

* Training: The `trainer.train([...])` command feeds the conversation into the chatbot's training process.

* Testing: After training, you can test it by getting a response from the bot using `chatbot.get_response()`.

This will allow the chatbot to respond based on the provided conversation, and you can expand it further with more phrases or different topics.

In [ ]:
## Training with Custom Data
from chatterbot.trainers import ListTrainer
trainer = ListTrainer(chatbot)
trainer.train([
"How are you?",
"I am good.",
"That is good to hear.",
"Thank you",
"You're welcome."
])

In [ ]:
## Testing the Custom Data
response = chatbot.get_response("How are you?")
print(response)

Want something even more fun? Then, you'll have to increase the size of the training library:

In [ ]:
# Initialize the chatbot
chatbot = ChatBot('FunBot')

# Set up the trainer
trainer = ListTrainer(chatbot)

# Train the chatbot with a more interesting and varied conversation
trainer.train([
    "How are you?",
    "I'm doing fantastic, thanks for asking!",
    "What’s up?",
    "Not much, just hanging out here with you!",
    "Tell me a joke.",
    "Why don't skeletons fight each other? They don't have the guts!",
    "What's your favorite color?",
    "I like blue, but I'm open to all colors!",
    "Do you like music?",
    "Absolutely! I love all kinds of music, especially upbeat tunes. How about you?",
    "What’s your favorite food?",
    "I don't eat, but I think pizza sounds like a great choice!",
    "Where are you from?",
    "I’m from the digital world, always connected and learning.",
    "Can you help me with math?",
    "Sure! What's the problem? I can add, subtract, multiply... well, you get the idea!",
    "What's the weather like?",
    "I can’t check the weather, but I’d imagine it’s perfect wherever you are!",
    "Thank you",
    "You're very welcome! If you need more help, just let me know.",
    "Tell me something interesting.",
    "Did you know that honey never spoils? Archaeologists have found pots of honey in ancient tombs, and it’s still good to eat!",
    "Goodbye",
    "Goodbye! Come back soon, I’ll be here!"
])

And now you can test it with a number of questions:

In [ ]:
# Test the chatbot with a variety of inputs
print(chatbot.get_response("How are you?"))
print(chatbot.get_response("Tell me a joke"))
print(chatbot.get_response("Can you help me with math?"))
print(chatbot.get_response("What's the weather like?"))
print(chatbot.get_response("Goodbye"))

# **4. Integrating Your Chatbot into a web page**

In order to run your chatbot from a web page, you need a number of components:

1. **A webserver:** The Flask code below sets up a basic web server that integrates a chatbot using the ChatterBot library. It defines two routes: the root route (`"/"`) renders an HTML page (`index.html`) that serves as the chat interface, and the `"/get"` route receives user input via a GET request and returns the chatbot's response. When a user types a message on the webpage, JavaScript sends it to the `/get` endpoint, where the chatbot processes the message and responds. The response is then displayed on the webpage, enabling real-time interaction between the user and the bot through a simple browser interface.

2. **A page:** You need a simple front-end interface for your chatbot. Below, we create a folder named templates in the same directory as your Flask app, and inside it, we create a file named index.html.

## **Step 1: Cleaning up Chatterbot for Google Colab**

In [ ]:
## Starting with a clean install of a Chatterbot version that This version avoids strict PyYAML version pinning and works fine in Colab.

!pip uninstall chatterbot -y
!pip install git+https://github.com/gunthercox/ChatterBot.git@master

In [ ]:
## Testing the install

from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

bot = ChatBot("Test Bot")
trainer = ChatterBotCorpusTrainer(bot)
trainer.train("chatterbot.corpus.english")

response = bot.get_response("Hello, how are you?")
print(response)

## **Step 2: Set Up a Simple Flask Web Interface with `flask-pyngrok`**

In order to set up a Flask webserver, you will need an ngrok authentication token. This requires the following steps:

* **Step 1:** Install pyngrok
* **Step 2:** Create an ngrok account (if you don't have one)
Go to: https://dashboard.ngrok.com/signup
* **Step 3:** Get your authtoken.

  After logging in, visit:
https://dashboard.ngrok.com/get-started/your-authtoken

  Copy your token — it looks like this:
`1h2g3exampleAuthtoken4u5x6y`

* **Step 4:** Install the authtoken in Colab and test connectivity (see below)

In [ ]:
## Step 1
!pip install pyngrok

In [ ]:
## Step 4

from pyngrok import ngrok

# Replace with your actual token
ngrok.set_auth_token("YOUR_AUTHTOKEN_HERE") # replace with your auth token; keep the ""
public_url = ngrok.connect(5000)
print(f"🔥 Public URL: {public_url}")

## **Step 3: Create the Chatbot Instance**

In [ ]:
from flask import Flask, render_template_string, request, jsonify
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from pyngrok import ngrok

# Create a Flask app
app = Flask(__name__)

# Create a chatbot instance
chatbot = ChatBot("ChatterBot")
trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train("chatterbot.corpus.english")

##**Step 4: Build the Web Front End**

In [ ]:
# HTML + JS frontend
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Chat with Bot</title>
    <style>
        body { font-family: Arial; background: #f4f4f4; padding: 30px; }
        #chatbox { background: white; border: 1px solid #ccc; padding: 15px; height: 400px; overflow-y: scroll; }
        .msg { margin: 10px 0; }
        .user { color: blue; }
        .bot { color: green; }
    </style>
</head>
<body>
    <h2>Chat with ChatterBot 🤖</h2>
    <div id="chatbox"></div>
    <input type="text" id="input" placeholder="Say something..." autofocus>
    <button onclick="send()">Send</button>

    <script>
        async function send() {
            let input = document.getElementById("input");
            let text = input.value;
            input.value = "";

            let chatbox = document.getElementById("chatbox");
            chatbox.innerHTML += `<div class='msg user'>You: ${text}</div>`;

            let res = await fetch("/get", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify({ message: text })
            });

            let data = await res.json();
            chatbox.innerHTML += `<div class='msg bot'>Bot: ${data.reply}</div>`;
            chatbox.scrollTop = chatbox.scrollHeight;
        }
    </script>
</body>
</html>
"""

# Routes
@app.route("/")
def home():
    return render_template_string(html_template)

@app.route("/get", methods=["POST"])
def get_bot_response():
    user_text = request.json["message"]
    response = str(chatbot.get_response(user_text))
    return jsonify(reply=response)

# Expose the app
ngrok.set_auth_token("YOUR_AUTHTOKEN_HERE")  # Replace with your real token; keep the ""
public_url = ngrok.connect(5000)
print(f"🔥 Chatbot is live at: {public_url}")

app.run(port=5000)

**What this does**:

* Starts a chatbot (trained on chatterbot.corpus.english)

* Serves a lightweight chat UI via Flask

* Uses JavaScript to send and receive messages without reloading

* Ngrok exposes it publicly so you can share the link or test on your phone

In the end, ysou should see a line saying

`Chatbot is live at: NgrokTunnel: "https://6d20-34-169-117-127.ngrok-free.app"`

or a similar link. Click on the link, and you should see your chatbot webpage. It will look like this:

<img src= "https://raw.githubusercontent.com/shstreuber/Data-Mining/refs/heads/master/images/Week14_Chatterbot.png">





#**4. Challenges and Solutions in Building Python AI Chatbots**

1. **Challenge 1: Understanding User Intent**
   
   **Problem:** One of the biggest challenges in chatbot development is accurately understanding user intent. As language can be ambiguous and context-dependent, deciphering what a user truly means can be complex.

   **Solution:** Utilize NLP techniques like Named Entity Recognition (NER) and Intent Classification to interpret user input. Leverage machine learning models trained on large datasets to better recognize and respond to varied user queries.

2. **Challenge 2: Handling Conversational Context**
   **Problem:*** Maintaining the context of a conversation is crucial for delivering coherent responses. Without this, the chatbot might not understand references to previous messages, leading to a disjointed conversation.

   **Solution:** Implement context management in your chatbot using techniques like dialogue management and session tracking. Libraries like Rasa provide tools for managing conversational context.

3. **Challenge 3: Dealing with Unfamiliar Queries**
   **Problem:** Chatbots, especially rule-based ones, might stumble upon unfamiliar or out-of-scope queries, which can disrupt the user experience.

   **Solution:** Train your chatbot to handle unfamiliar queries gracefully. This could involve directing users to human support or suggesting alternate queries. Additionally, incorporate regular updates and training to your chatbot based on new and trending queries.

4. **Challenge 4: Lack of Personalization**
   **Problem:** Generic responses can make interactions with a chatbot feel mechanical and impersonal, diminishing user engagement.

   **Solution:** Implement personalization in your chatbot. This could range from using the user's name to tailoring responses based on user preferences and past interactions.

5. **Challenge 5: Scaling and Deployment**
   **Problem:** As your chatbot gets more complex and traffic increases, it may face issues related to performance, scalability, and deployment.

   **Solution:** Plan for scalability from the get-go. Utilize scalable cloud services and robust deployment practices. Monitor performance regularly and optimize as needed.

Remember, overcoming these challenges is part of the journey of developing a successful chatbot. Each challenge presents an opportunity to learn and improve, ultimately leading to a more sophisticated and engaging chatbot.